<div align="center"> 

**hopa1029, klfl1012**

<img src="s_purple.png" alt="drawing" width="2000" height= 200/>


<div align="left">

# Übungsblatt 2 - Personalabteilung
__________________________________________________________________________________________________________________________________________________________________________________________________________________

**DSCB310 – Datenanalyse und Business Intelligence 1**  

Wintersemester 22/23

##### Gliederung des Notebooks:

1. Ziel der Datenanaylse für die Personalabteilung
   
2. Fragestellungen:

   - P1: Prüfen Sie, ob ein Zusammenhang zwischen dem Trinkgeldgeben und dem Vorhandensein eines der folgenden Produkte in einer Bestellung existiert: 24852, 24964, 2120
  
   - P2: Lassen sich regionale Unterschiede im Trinkgeldverhalten erkennen?
  
   - P3: Welche Attribute einer Bestellung wirken sich auf das Trinkgeldverhalten aus?
  
   - P4: Spielt die Vergangenheit eines Users eine Rolle in Hinblick auf die Trinkgeldwahrscheinlichkeit, oder kommt es nur auf Inhalt und Parameter der aktuellen Bestellung an?
  
3. Zusätzliche Fragestellungen

##### Imports, Datensatz und Funktionen: 

**Imports:**

In [355]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from urllib.request import urlopen
import json

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import make_pipeline

**Datensatz:**

- Der verwendete Datensatz umfasst ungefähr 6.1 Millionen Bestellungen von Usern aus allen Counties von Californien bis auf Butte
    
- Ein Zeile entsrpicht damit einem Produkt und Bestellungen können sich somit über mehrere Zeilen im Datensatz ziehen
  
- Der Datensatz hat in der Spalte days_since_prior_order Nan-Values für jeweils die erste Bestellung jedes Users
 
- Zudem lässt sich sagen, dass Analysen zu den Counties San Bernardino, Placer und Trinity, welche 40, 48 und 130 Bestellungen besitzen, augrund dieser geringen Bestellugnsanzahl mit Vorsicht zu gewichten sind 

In [356]:
df = pd.read_parquet("DSCB310 - UE2 - Shopping Carts.parquet")

In [357]:
df.reset_index(drop= True).head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county
0,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo
1,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo
2,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo
3,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo
4,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo


In [358]:
# Einheitliche Print Funktion zur Rückgabe von accuracy, recall und precision des verwendeten Modells

def print_evaluation(pipeline_or_model: str, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred):
    accurary_train = accuracy_score(y_train, y_train_pred)
    recall_train = recall_score(y_train, y_train_pred)
    precision_train = precision_score(y_train, y_train_pred)

    accurary_test = accuracy_score(y_test, y_test_pred)
    recall_test = recall_score(y_test, y_test_pred)
    precision_test = precision_score(y_test, y_test_pred)
    
    print(
        f"{pipeline_or_model} Evaluation:\n"
        f"{'':6} {'ACC':>10} | {'REC':>14} | {'PRE':>10} | {'rows':>8} | {'columns':>8}\n"
        f"{'Train':6} {accurary_train:10.5f} | {recall_train:14.5f} | {precision_train:10.5f} | {X_train.shape[0]:8} | {X_train.shape[1]:8}\n"
        f"{'Test':6} {accurary_test:10.5f} | {recall_test:14.5f} | {precision_test:10.5f} | {X_test.shape[0]:8} | {X_test.shape[1]:8}\n"
    )

## 1. Ziel der Datenanalyse für die Personalabteilung:

- Nachvollziehbare und systematische Muster bei der Trinkgeldvergabe finden

- Wovon die Wahrscheinlichkeit, bei einer Bestellung Trinkgeld zu erhalten, abhängt

-> Damit die Chancengleichheit der Fahrer verbessert wird, indem sie entweder vom Unternehmen extra entlohnt werden oder die vielversprechenden und weniger aussichtsreichen Fahrten gleichmäßig auf die Fahrer verteilt werden können.

## 2. Fragestellungen:

### **P1:** Prüfen Sie, ob ein Zusammenhang zwischen dem Trinkgeldgeben und dem Vorhandensein eines der folgenden Produkte in einer Bestellung existiert: 24852, 24964, 2120:

In [359]:
df_tip = df.copy()

In [360]:
df_tip.loc[df_tip.product_id.isin([24852, 24964, 2120])].product_name.unique()

array(['Organic Garlic', 'Banana', 'Sauvignon Blanc'], dtype=object)

##### Bestellungen mit Organic Garlic

In [361]:
df_tip_1 = df.copy()

In [362]:
df_tip_1["p1_products_in_order"] = 0
df_tip_1.loc[df_tip_1.product_id.isin([24852]), "p1_products_in_order"] = 1

**Bestellungen die das Produkt Organic Garlic enthalten haben eine höhere Wahrscheinlichkeit einen Tipp zu erhalten**  

In [363]:
pd.crosstab(df_tip_1.p1_products_in_order, df.tip, margins= True, normalize= "index")

tip,0,1
p1_products_in_order,,
0,0.553860,0.446140
1,0.460500,0.539500
All,0.552504,0.447496


**Testen ob Unabhängig / Abhängig:**

- H0: (Nullhypothese) Die beiden Variablen sind unabhängig.
- H1: (alternative Hypothese) Die beiden Variablen sind nicht unabhängig.

In [364]:
pd.crosstab(df_tip_1.p1_products_in_order, df.tip, margins= True)

tip,0,1,All
p1_products_in_order,,,
0,3347279,2696263,6043542
1,41043,48084,89127
All,3388322,2744347,6132669


In [365]:
import scipy.stats as stats

data = pd.crosstab(df_tip_1.p1_products_in_order, df.tip)

stats.chi2_contingency(data)

(3095.9790391918004,
 0.0,
 1,
 array([[3339079.00728443, 2704462.99271557],
        [  49242.99271557,   39884.00728443]]))

Da der p-Wert des Tests weniger als 0.05 beträgt wird die Nullhypothese abgelehnt, es gilt H1.

In [366]:
pd.crosstab(df_tip_1.p1_products_in_order, df.tip, margins= True, normalize= "all")

tip,0,1,All
p1_products_in_order,,,
0,0.545811,0.439656,0.985467
1,0.006693,0.007841,0.014533
All,0.552504,0.447496,1.000000


In [367]:
# hji = hi * hj
pd.crosstab(df_tip_1.p1_products_in_order, df.tip, margins= True, normalize= "all").iloc[1, 1:2].values[0] == 0.006693 * 	0.007841

False

##### Bestellungen mit Banana

In [368]:
df_tip_2 = df.copy()

In [369]:
df_tip_2["p1_products_in_order"] = 0
df_tip_2.loc[df_tip_2.product_id.isin([24964]), "p1_products_in_order"] = 1

**Bestellungen die das Produkt Banana enthalten haben eine höhere Wahrscheinlichkeit einen Tipp zu erhalten**  

In [370]:
pd.crosstab(df_tip_2.p1_products_in_order, df.tip, margins= True, normalize= "index")

tip,0,1
p1_products_in_order,,
0,0.552415,0.447585
1,0.578847,0.421153
All,0.552504,0.447496


**Testen ob Unabhängig / Abhängig:**

- H0: (Nullhypothese) Die beiden Variablen sind unabhängig.
- H1: (alternative Hypothese) Die beiden Variablen sind nicht unabhängig.

In [371]:
pd.crosstab(df_tip_2.p1_products_in_order, df.tip, margins= True)

tip,0,1,All
p1_products_in_order,,,
0,3376429,2735694,6112123
1,11893,8653,20546
All,3388322,2744347,6132669


In [372]:
import scipy.stats as stats

data = pd.crosstab(df_tip_2.p1_products_in_order, df.tip)

stats.chi2_contingency(data)

(57.7583360634705,
 2.9638135633651274e-14,
 1,
 array([[3376970.26002969, 2735152.73997031],
        [  11351.73997031,    9194.26002969]]))

Da der p-Wert des Tests weniger als 0.05 beträgt wird die Nullhypothese abgelehnt, es gilt H1.

In [373]:
pd.crosstab(df_tip_2.p1_products_in_order, df.tip, margins= True, normalize= "all")

tip,0,1,All
p1_products_in_order,,,
0,0.550564,0.446085,0.99665
1,0.001939,0.001411,0.00335
All,0.552504,0.447496,1.00000


In [374]:
# hji = hi * hj
pd.crosstab(df_tip_2.p1_products_in_order, df.tip, margins= True, normalize= "all").iloc[1, 1:2].values[0] == 0.447496 * 0.018123

False

##### Bestellungen mit Sauvignon Blanc

In [375]:
df_tip_3 = df.copy()

In [376]:
df_tip_3["p1_products_in_order"] = 0
df_tip_3.loc[df_tip_3.product_id.isin([2120]), "p1_products_in_order"] = 1

**Bestellungen die das Produkt Sauvignon Blanc enthalten haben eine höhere Wahrscheinlichkeit einen Tipp zu erhalten**  

In [377]:
pd.crosstab(df_tip_3.p1_products_in_order, df.tip, margins= True, normalize= "index")

tip,0,1
p1_products_in_order,,
0,0.552599,0.447401
1,0.156929,0.843071
All,0.552504,0.447496


**Testen ob Unabhängig / Abhängig:**

- H0: (Nullhypothese) Die beiden Variablen sind unabhängig.
- H1: (alternative Hypothese) Die beiden Variablen sind nicht unabhängig.

In [378]:
pd.crosstab(df_tip_3.p1_products_in_order, df.tip, margins= True)

tip,0,1,All
p1_products_in_order,,,
0,3388091,2743106,6131197
1,231,1241,1472
All,3388322,2744347,6132669


In [379]:
import scipy.stats as stats

data = pd.crosstab(df_tip_3.p1_products_in_order, df.tip)

stats.chi2_contingency(data)

(930.2450959070047,
 2.6122802664072974e-204,
 1,
 array([[3.38750871e+06, 2.74368829e+06],
        [8.13285371e+02, 6.58714629e+02]]))

Da der p-Wert des Tests weniger als 0.05 beträgt wird die Nullhypothese abgelehnt, es gilt H1.

In [380]:
pd.crosstab(df_tip_3.p1_products_in_order, df.tip, margins= True, normalize= "all")

tip,0,1,All
p1_products_in_order,,,
0,0.552466,0.447294,0.99976
1,0.000038,0.000202,0.00024
All,0.552504,0.447496,1.00000


In [381]:
# hji = hi * hj
pd.crosstab(df_tip_3.p1_products_in_order, df.tip, margins= True, normalize= "all").iloc[1, 1:2].values[0] == 0.447496 * 0.018123

False

##### Bestellungen mit Organic Garlic, Banana und Sauvignon Blanc

In [382]:
df_tip["p1_products_in_order"] = 0
df_tip.loc[df_tip.product_id.isin([24852, 24964, 2120]), "p1_products_in_order"] = 1

**Bestellungen die die Produkte Organic Garlic, Banana und Sauvignon Blanc enthalten haben eine höhere Wahrscheinlichkeit einen Tipp zu erhalten**  

In [383]:
pd.crosstab(df_tip.p1_products_in_order, df.tip, margins= True, normalize= "index")

tip,0,1
p1_products_in_order,,
0,0.553872,0.446128
1,0.478357,0.521643
All,0.552504,0.447496


**Testen ob Unabhängig / Abhängig:**

- H0: (Nullhypothese) Die beiden Variablen sind unabhängig.
- H1: (alternative Hypothese) Die beiden Variablen sind nicht unabhängig.

In [384]:
pd.crosstab(df_tip.p1_products_in_order, df.tip, margins= True)

tip,0,1,All
p1_products_in_order,,,
0,3335155,2686369,6021524
1,53167,57978,111145
All,3388322,2744347,6132669


In [385]:
import scipy.stats as stats

data = pd.crosstab(df_tip.p1_products_in_order, df.tip)

stats.chi2_contingency(data)

(2516.7353829996982,
 0.0,
 1,
 array([[3326913.98194294, 2694610.01805706],
        [  61408.01805706,   49736.98194294]]))

Da der p-Wert des Tests weniger als 0.05 beträgt wird die Nullhypothese abgelehnt, es gilt H1.

In [386]:
pd.crosstab(df_tip.p1_products_in_order, df.tip, margins= True, normalize= "all")

tip,0,1,All
p1_products_in_order,,,
0,0.543834,0.438042,0.981877
1,0.008669,0.009454,0.018123
All,0.552504,0.447496,1.000000


In [387]:
# hji = hi * hj
pd.crosstab(df_tip.p1_products_in_order, df.tip, margins= True, normalize= "all").iloc[1, 1:2].values[0] == 0.447496 * 0.018123

False

0.447496 * 0.018123 = 0.00811 != 0.009454

**Dadurch ist gezeigt, dass die Tippwahrscheinlichkeit der Bestellungen davon abhängig ist, ob sie die Produkte: Organic Garlic, Banana und Sauvignon Blanc enthält.**  
**Die Bestelungen die die Produkte enthalten, erhalten statistisch gesehen mehr Trinkgeld.** 



### **P2:** Lassen sich regionale Unterschiede im Trinkgeldverhalten erkennen?

In [388]:
df_p2 = df.copy()

- Durchschnittliche Wahrscheinlichkeit das eine Bestellung Trinkgeld erhält, pro County berechnen

In [389]:
df_county_tip1 = df_p2.groupby(["county", "order_id"]).agg(tip = ("tip", "mean")).reset_index()
df_county_tip2 = df_county_tip1.groupby("county").agg(tip = ("tip", "mean"), orders= ("order_id", "count")).sort_values(by= "tip", ascending= False).reset_index()
df_county_tip2["orders"] = df_county_tip2.orders / df_county_tip2.orders.sum()

In [483]:
df_county_tip2.head(10)

,county,tip,orders
0,San Diego,0.530895,0.003920
1,Santa Barbara,0.507548,0.023030
2,Lassen,0.502494,0.003964
3,Del Norte,0.501239,0.003323
4,Modoc,0.492131,0.015599
5,Tuolumne,0.483859,0.004287
6,San Benito,0.475038,0.012046
7,Shasta,0.466202,0.006143
8,El Dorado,0.465879,0.002511
9,San Francisco,0.462101,0.021173


**Plotten der Tippwahrscheinlichkeit pro County:**

In [391]:
# Plot P2: Tippwahrscheinlichkeit pro County

fig1 = make_subplots(cols= 1, rows= 2, shared_xaxes= True, shared_yaxes= False, vertical_spacing= 0)

for county in df_county_tip2.county:

 if df_county_tip2[df_county_tip2.county == county].tip.values >= 0.5:
  color = px.colors.sequential.Purpor[5]
  text= df_county_tip2.loc[df_county_tip2.county == county].tip.values.round(3)

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].tip,
    name= f"{county}: {(df_county_tip2.loc[df_county_tip2.county == county].tip.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{county}",
    hovertemplate = f"<b>{county}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].orders,
    name= f"Orders: {(df_county_tip2.loc[df_county_tip2.county == county].orders.values[0] * 100).round(3)}%",
    base = -df_county_tip2.loc[df_county_tip2.county == county].orders,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{county}",
    hovertemplate = f"<b>{county}</b><br><br>" +
                "Orders: %{base}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 2
  )

 elif 0.4 < df_county_tip2.loc[df_county_tip2.county == county].tip.values < 0.5:
  color= px.colors.sequential.Greys[3]

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].tip,
    name= f"{county}: {(df_county_tip2.loc[df_county_tip2.county == county].tip.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{county}",
    hovertemplate = f"<b>{county}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].orders,
    name= f"Orders: {(df_county_tip2.loc[df_county_tip2.county == county].orders.values[0] * 100).round(3)}%",
    base = -df_county_tip2.loc[df_county_tip2.county == county].orders,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{county}",
    hovertemplate = f"<b>{county}</b><br><br>" +
                "Orders: %{base}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 2
  )

 elif 0.3 < df_county_tip2.loc[df_county_tip2.county == county].tip.values < 0.4:
  color= px.colors.sequential.Greys[2]

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].tip,
    name= f"{county}: {(df_county_tip2.loc[df_county_tip2.county == county].tip.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{county}",
    hovertemplate = f"<b>{county}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].orders,
    name= f"Orders: {(df_county_tip2.loc[df_county_tip2.county == county].orders.values[0] * 100).round(3)}%",
    base = -df_county_tip2.loc[df_county_tip2.county == county].orders,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{county}",
    hovertemplate = f"<b>{county}</b><br><br>" +
                "Orders: %{base}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 2
  )

 elif 0.2 < df_county_tip2.loc[df_county_tip2.county == county].tip.values < 0.3:
  color= px.colors.sequential.Greys[1]
  text= df_county_tip2.loc[df_county_tip2.county == county].tip.values.round(3)

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].tip,
    name= f"{county}: {(df_county_tip2.loc[df_county_tip2.county == county].tip.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{county}",
    hovertemplate = f"<b>{county}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].orders,
    name= f"Orders: {(df_county_tip2.loc[df_county_tip2.county == county].orders.values[0] * 100).round(3)}%",
    base = -df_county_tip2.loc[df_county_tip2.county == county].orders,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{county}",
    hovertemplate = f"<b>{county}</b><br><br>" +
                "Orders: %{base}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 2
  )

fig1.add_hline(
 y= 0.5,
 opacity= 0.3,
 line_dash= "solid",
 annotation_text= 0.5,
 annotation_position= "top right",
 line_color= px.colors.sequential.Greys[7],
 annotation_font_color= px.colors.sequential.Greys[7],
 annotation_font_size= 12,
 layer= "below",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= 0.3,
 opacity= 0.3,
 line_dash= "solid",
 annotation_text= 0.3,
 annotation_position= "top right",
 line_color= px.colors.sequential.Greys[7],
 annotation_font_color= px.colors.sequential.Greys[7],
 annotation_font_size= 12,
 layer= "below",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= -0.05,
 opacity= 0.3,
 line_dash= "solid",
 annotation_text= -0.05,
 annotation_position= "top right",
 line_color= px.colors.sequential.Greys[7],
 annotation_font_color= px.colors.sequential.Greys[7],
 annotation_font_size= 12,
 layer= "below",
 col= 1,
 row= 2
)

fig1.add_hline(
 y= -0.03,
 opacity= 0.3,
 line_dash= "solid",
 annotation_text= -0.03,
 annotation_position= "top right",
 line_color= px.colors.sequential.Greys[7],
 annotation_font_color= px.colors.sequential.Greys[7],
 annotation_font_size= 12,
 layer= "below",
 col= 1,
 row= 2
)

fig1.add_hline(
 y= 0,
 opacity= 0.2,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= 0,
 opacity= 0.2,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 layer= "below",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 layer= "below",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= -0.075,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 2
)

fig1.update_annotations(opacity= 0.8)
fig1.update_layout(title= dict(text= "<b>Regionale Unterschiede im Trinkgeldverhalten</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), legend_title_text = "<b>Counties</b>", height= 700)
fig1.update_xaxes(title_text="<b>Counties</b>", ticks= "outside", col= 1, row= 2, showline= True)
fig1.update_xaxes(showline= False, col= 1, row= 1)
fig1.update_yaxes(title_text="<b>Tipp Wshk.</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 1, tickvals= [0, 0.1, 0.2, 0.3, 0.4, 0.5], showline= False)
fig1.update_yaxes(title_text="<b>Order Anteil</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 2, tickvals= [0, -0.01, -0.03, -0.05, -0.07])
fig1.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))
fig1.show()

In [392]:
# Plot P3: Tippwahrscheinlichkeit pro County
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
  go.Scatter(
    x= df_county_tip2.county,
    y= df_county_tip2.orders,
    mode= "lines+markers", 
    marker= dict(color= px.colors.sequential.Greys[4], size= 4),
    name= "Bestellungen",
    fill= "tozeroy",
    hoverinfo = "y"
  ),
  secondary_y= True
)


for county in df_county_tip2.county:

 if df_county_tip2[df_county_tip2.county == county].tip.values >= 0.5:
  color = px.colors.sequential.Purpor[5]
  text= df_county_tip2.loc[df_county_tip2.county == county].tip.values.round(2)

  fig.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].tip,
    name= f"Tipp: {(df_county_tip2.loc[df_county_tip2.county == county].tip.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{county}",
    legendgrouptitle_text= f"{county}",
    hovertemplate = f"<b>{df_county_tip2.loc[df_county_tip2.county == county].county.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   secondary_y= False
  )

 elif 0.4 <= df_county_tip2.loc[df_county_tip2.county == county].tip.values < 0.5:
  color= px.colors.sequential.Greys[3]

  fig.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].tip,
    name= f"Tipp: {(df_county_tip2.loc[df_county_tip2.county == county].tip.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{county}",
    legendgrouptitle_text= f"{county}",
    hovertemplate = f"<b>{df_county_tip2.loc[df_county_tip2.county == county].county.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   secondary_y= False
  )

 elif 0.3 < df_county_tip2.loc[df_county_tip2.county == county].tip.values < 0.4:
  color= px.colors.sequential.Greys[2]

  fig.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].tip,
    name= f"Tipp: {(df_county_tip2.loc[df_county_tip2.county == county].tip.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{county}",
    legendgrouptitle_text= f"{county}",
    hovertemplate = f"<b>{df_county_tip2.loc[df_county_tip2.county == county].county.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   secondary_y= False
  )

 elif 0.2 < df_county_tip2.loc[df_county_tip2.county == county].tip.values < 0.3:
  color= px.colors.sequential.Greys[1]
  text= df_county_tip2.loc[df_county_tip2.county == county].tip.values.round(2)

  fig.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].tip,
    name= f"Tipp: {(df_county_tip2.loc[df_county_tip2.county == county].tip.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{county}",
    legendgrouptitle_text= f"{county}",
    hovertemplate = f"<b>{df_county_tip2.loc[df_county_tip2.county == county].county.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   secondary_y= False
  )


fig.update_layout(title= dict(text= "<b>Tippwahrscheinlichkeit pro County</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), legend_title_text = "<b>Tage</b>", height= 650)
fig.update_xaxes(title_text="<b>Tage</b>", ticks= "outside")
fig.update_yaxes(title_text="<b>Tipp Wshk.</b>", ticks= "outside", showgrid= True, gridcolor= px.colors.sequential.Greys[1])
fig.update_yaxes(title_text="<b>Order Anteil</b>", ticks= "outside", showgrid= False, secondary_y= True, showline= True, linecolor= "black")
fig.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))
fig.show()

In [393]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    geojsn = json.load(response)

In [394]:
a = geojsn["features"].__len__()
for i in range(a,0,-1):
    if geojsn["features"][i-1]["properties"]["STATE"] != "06":
        geojsn["features"].pop(i-1)

In [395]:
# Plot P2: Regionale Unterschiede im Tippverhalten Map

colorscales = [(0,px.colors.sequential.Greys[0]),
                (0.3, px.colors.sequential.Greys[2]),
                (0.6, px.colors.sequential.Purp[3]),
                (0.8, px.colors.sequential.Purp[5]),
                (1,px.colors.sequential.Purpor[5])]


fig = make_subplots(cols= 1, rows= 3, specs = [[dict(type= "choroplethmapbox", rowspan= 2)],[dict(type= "choroplethmapbox", rowspan= 2)], [dict(type= "table")]], vertical_spacing = 0.03, shared_xaxes= True)

fig.add_trace(
    go.Choroplethmapbox(
        geojson= geojsn,
        featureidkey= "properties.NAME",
        locations= df_county_tip2.county,
        z= df_county_tip2.tip,
        showlegend=False,
        showscale=True,
        marker= dict(line= dict(width= 0.2, color= "black"), opacity= 1),
        colorscale = colorscales
    ),
    col= 1,
    row= 1
)

fig.add_trace(
    go.Table(
        columnorder= [1, 2, 3, 4, 5],
        columnwidth= [1, 1, 1, 1, 1],
        header= dict(
            values= list(df_county_tip2.columns),
            fill_color= px.colors.sequential.Purp[3],
            line_color= px.colors.sequential.Greys[4],
            font= dict(color= "white", size=15),
            height= 35
        ),
        cells= dict(
            values= df_county_tip2.round(3).T.values,
            line_color= px.colors.sequential.Greys[4],
            fill_color= "white",
            font= dict(color= "black", size=13),
            align= "center",
            height= 30
        )
    ),
    col= 1,
    row= 3
)

fig.update_layout(mapbox_style="carto-positron",
                    mapbox_zoom= 4.3,
                    mapbox_center = {"lat": 37, "lon": -120},
                    height= 700,
                    title= dict(text= "<b>Regionale Unterschiede im Trinkgeldverhalten</b>",
                                font_size= 20,
                                y= 0.99,
                                x= 0.45
                            )
                )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

- Trinkgeldwahrscheinlichkeit bewegt sich im Bereich von 28.2% in Tulare bis 53.1% in San Diego 

- San Bernardino, Placer und Trinity haben mit 0,0066%, 0.0076% und 0.0305% einen zu geringen Anteil am Bestelldatensatz, weshalb sie für Aussagen nicht berücksichtig werden

### **P3:** Welche Attribute einer Bestellung wirken sich auf das Trinkgeldverhalten aus?

In [396]:
df_p3 = df.copy()

#### Korrelationsanalyse

In [397]:
df_p3[["product_id", "add_to_cart_order",	"reordered",	"order_number",	"order_dow",	"order_hour_of_day",	"days_since_prior_order",	"product_name",	"aisle_id",	"department_id",	"county", "tip"]].corr()

,product_id,add_to_cart_order,reordered,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id,tip
product_id,1.000000,0.005204,0.003051,-0.001738,-0.002709,0.001654,0.000652,0.002292,-0.026906,-0.001380
add_to_cart_order,0.005204,1.000000,-0.132393,-0.005447,-0.007274,-0.012353,0.055913,0.008351,0.028911,0.052247
reordered,0.003051,-0.132393,1.000000,0.300142,-0.011324,-0.022744,-0.130482,0.005835,-0.038138,0.077467
order_number,-0.001738,-0.005447,0.300142,1.000000,0.012476,-0.040606,-0.357631,0.001855,0.002189,0.093831
order_dow,-0.002709,-0.007274,-0.011324,0.012476,1.000000,0.009725,-0.030760,-0.002564,0.003648,-0.114064
order_hour_of_day,0.001654,-0.012353,-0.022744,-0.040606,0.009725,1.000000,0.004329,-0.000540,-0.007007,0.025336
days_since_prior_order,0.000652,0.055913,-0.130482,-0.357631,-0.030760,0.004329,1.000000,0.004795,-0.000918,-0.143219
aisle_id,0.002292,0.008351,0.005835,0.001855,-0.002564,-0.000540,0.004795,1.000000,0.061251,0.001030
department_id,-0.026906,0.028911,-0.038138,0.002189,0.003648,-0.007007,-0.000918,0.061251,1.000000,0.046344
tip,-0.001380,0.052247,0.077467,0.093831,-0.114064,0.025336,-0.143219,0.001030,0.046344,1.000000


- Werte sind alle < 0.15, daher lässt sich über die Korrelation keine ausreichendere Aussage zu der Frage treffen  

#### Machine Learning Modell und Feature Importance:

- Da mit der Korrelation keine richtige Aussage getroffen werden konnte, wird anhand der Feature Importance eines Modelles versucht die Attribute zu finden, die die Trinkgeldwahrscheinlichkeit am stärsten beeinflussen
  
- Dabei werden nur 10% des gesamt Datensatz verwendet, um Rechenzeit einzusparen
  
- Anschließend wird ein Entscheidungsbaum auf diesem Sample trainiert und die Feature Importance der einzelene Attribute des Datensatzes bestimmt und diese dann genauer analysiert

##### Preprocessing

In [398]:
df_ml = df_p3.reset_index(drop= True)

**Sampling des Datensatzes auf 10% um Rechenzeit zu ersparen**

In [399]:
df_sample = df_ml.sample(frac= 0.1)

In [400]:
df_classifier = df_sample.copy()

**One Hot Encoding der Spalten County und days_since_prior_order zum weiterverarbeiten**

In [401]:
df_classifier = pd.get_dummies(df_classifier, columns= ["county", "days_since_prior_order"], dummy_na= True)

In [402]:
X = df_classifier.drop(["product_name", "department", "aisle", "tip"], axis= 1)
y = df_classifier["tip"]

In [403]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=0)

##### **DTC**

- Mittels einer 5-Fachen Kreuzvalidierung und Hyperparametertuning wird die beste Kombination aus Baumtiefe (max_depth) und minimaler Anzahl an benötigter Beispiele für einen Split (min_samples) gesucht:

In [404]:
gs_crss_dtc = GridSearchCV(
 estimator= DecisionTreeClassifier(),
 param_grid= dict(min_samples_split= [2, 10, 20, 50, 100], max_depth= [1, 3, 5, 9], random_state= [0]),
 scoring= "accuracy",
 n_jobs= -1
).fit(X_train, y_train)

print(f"Best param: {gs_crss_dtc.best_params_} with acc: {gs_crss_dtc.best_score_}")

Best param: {'max_depth': 9, 'min_samples_split': 2, 'random_state': 0} with acc: 0.6145740128336348


In [405]:
best_dtc = DecisionTreeClassifier(max_depth= 9, min_samples_split= 2, random_state= 0).fit(X_train, y_train)

y_best_train_pred = best_dtc.predict(X_train)
y_best_test_pred = best_dtc.predict(X_test)

print_evaluation("DTC", X_train, X_test, y_train, y_test, y_best_train_pred, y_best_test_pred)

DTC Evaluation:
              ACC |            REC |        PRE |     rows |  columns
Train     0.61907 |        0.46175 |    0.59685 |   490613 |      100
Test      0.61140 |        0.45428 |    0.58617 |   122654 |      100



In [406]:
feature_importances = pd.DataFrame.from_dict(dict(zip(X.columns, best_dtc.feature_importances_)), orient= "index", columns= ["feature_importance"])

df_feature_imp = feature_importances.reset_index().rename(columns= {"index": "feature"})

In [407]:
df_feature_imp = df_feature_imp.sort_values(by= "feature_importance", ascending= True).tail(9)
df_feature_imp

,feature,feature_importance
0,order_id,0.018467
2,add_to_cart_order,0.023052
4,user_id,0.026026
9,department_id,0.047558
75,days_since_prior_order_7.0,0.055693
7,order_hour_of_day,0.127835
98,days_since_prior_order_30.0,0.166785
5,order_number,0.191851
6,order_dow,0.279592


**Es ergibt sich folgende Features Importance für die Feature in unserem Datensatz:**

In [408]:
# P3 Plot: Feature Importance

fig = go.Figure()


for feature in df_feature_imp.feature.unique():

 if df_feature_imp.loc[df_feature_imp.feature == feature].feature_importance.values >= 0.1:
  color= px.colors.sequential.Purpor[5]

  fig.add_trace(
   go.Bar(
    x= df_feature_imp.loc[df_feature_imp.feature == feature].feature_importance,
    y= df_feature_imp.loc[df_feature_imp.feature == feature].feature,
    marker= dict(color= color),
    orientation = "h",
    text= df_feature_imp.loc[df_feature_imp.feature == feature].feature_importance.round(2),
    showlegend= False,
    width= 0.8,
    hovertemplate= f"<b>{feature}</b><br><br>" +
                   "feature_importance: %{x}<br>" +
                   "<extra></extra>"
    )
  )
 else:
  color= px.colors.sequential.Greys[3]

  fig.add_trace(
   go.Bar(
    x= df_feature_imp.loc[df_feature_imp.feature == feature].feature_importance,
    y= df_feature_imp.loc[df_feature_imp.feature == feature].feature,
    marker= dict(color= color),
    orientation = "h",
    text= df_feature_imp.loc[df_feature_imp.feature == feature].feature_importance.round(3),
    showlegend= False,
    width= 0.8,
    hovertemplate= f"<b>{feature}</b><br><br>" +
                   "feature_importance: %{x}<br>" +
                   "<extra></extra>"
    )
  )

fig.update_layout(title= dict(text= "<b>Feature Importance der Attribute</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), height= 600)
fig.update_xaxes(title_text="<b>Importance</b>", ticks= "outside")
fig.update_xaxes(showline= True)
fig.update_yaxes(title_text="<b>Feature</b>", ticks= "outside", showgrid= True, gridcolor= "grey", showline= True)
fig.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))
fig.show()

**Anhand der neu gewonnen Information, welche unserer Feature die Trinkgeldwahrscheinlichkeit am meisten beeinflussen, werden diese nun näher untersucht**

#### Attribut: order_dow

- Gruppieren der Bestelldaten nach Wochentagen und anschließend berechnen der Trinkgeldwahrscheinlichkeit  

In [409]:
df1 = df_p3.groupby(["order_dow", "order_id"], as_index= False).agg(tip_orders_per_day = ("tip", "mean"))
df1.head()

,order_dow,order_id,tip_orders_per_day
0,0,4,1.0
1,0,6,0.0
2,0,9,1.0
3,0,10,0.0
4,0,19,1.0


In [410]:
df_order_dow= df1.groupby("order_dow", as_index= False).agg(tip = ("tip_orders_per_day","sum"), orders= ("order_id", "nunique"))
df_order_dow["tip_norm"] = df_order_dow.tip / df_order_dow.orders
df_order_dow["orders_norm"] = df_order_dow.orders / df_order_dow.orders.sum()
df_order_dow.head()

,order_dow,tip,orders,tip_norm,orders_norm
0,0,53749.0,106842,0.503070,0.176041
1,1,54928.0,104713,0.524558,0.172533
2,2,30614.0,83177,0.368058,0.137049
3,3,27884.0,77329,0.360589,0.127413
4,4,27780.0,75323,0.368812,0.124108


In [411]:
df_order_dow["order_dow_day"] = ["Montag", "Dienstag", "Mittwoch", "Donnerstag", "Freitag", "Samstag", "Sonntag"]
df_order_dow.head()

,order_dow,tip,orders,tip_norm,orders_norm,order_dow_day
0,0,53749.0,106842,0.503070,0.176041,Montag
1,1,54928.0,104713,0.524558,0.172533,Dienstag
2,2,30614.0,83177,0.368058,0.137049,Mittwoch
3,3,27884.0,77329,0.360589,0.127413,Donnerstag
4,4,27780.0,75323,0.368812,0.124108,Freitag


**Plotten der Tippwahrscheinlichkeit pro Wochentag**

In [412]:
# Plot P3: Order_dow

df_order_dow["tip_norm"] = df_order_dow.tip_norm.round(3)
df_order_dow["orders_norm"] = df_order_dow.orders_norm.round(3)

fig = make_subplots(cols= 1, rows=2, specs= [[dict(type= "table")], [dict(type= "bar")]], vertical_spacing= 0.1)

fig.add_trace(
  go.Table(
    columnorder= [1, 2, 3, 4, 5],
    columnwidth= [1, 1, 1, 1, 1],
    header=dict(
        values=list(df_order_dow.columns),
        fill_color= px.colors.sequential.Purpor[5],
        line_color= px.colors.sequential.Greys[2],
        font= dict(color= "white", size=15),
        height= 35
        ),
    cells=dict(
        values=df_order_dow.T.values,
        fill_color= "white",
        line_color= px.colors.sequential.Greys[2],
        font= dict(color= "black", size=13),
        align= "center",
        height= 30
        )
    ),
    col= 1,
    row= 1
)

for order_dow in df_order_dow.order_dow:

 if df_order_dow[df_order_dow.order_dow == order_dow].tip_norm.values >= 0.5:
  color = px.colors.sequential.Purpor[5]
  text= df_order_dow.loc[df_order_dow.order_dow == order_dow].tip_norm.values.round(3)

  fig.add_trace(
   go.Bar(
    x= df_order_dow.loc[df_order_dow.order_dow == order_dow].order_dow,
    y= df_order_dow.loc[df_order_dow.order_dow == order_dow].tip_norm,
    name= f"Tipp: {(df_order_dow.loc[df_order_dow.order_dow == order_dow].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{order_dow}",
    legendgrouptitle_text = f"{order_dow}: {df_order_dow.loc[df_order_dow.order_dow == order_dow].order_dow_day.values[0]}",
    hovertemplate = f"<b>{df_order_dow.loc[df_order_dow.order_dow == order_dow].order_dow_day.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 2
  )

 else:
  color= px.colors.sequential.Greys[2]

  fig.add_trace(
   go.Bar(
    x= df_order_dow.loc[df_order_dow.order_dow == order_dow].order_dow,
    y= df_order_dow.loc[df_order_dow.order_dow == order_dow].tip_norm,
    name= f"Tipp: {(df_order_dow.loc[df_order_dow.order_dow == order_dow].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{order_dow}",
    legendgrouptitle_text = f"{order_dow}: {df_order_dow.loc[df_order_dow.order_dow == order_dow].order_dow_day.values[0]}",
    hovertemplate = f"<b>{df_order_dow.loc[df_order_dow.order_dow == order_dow].order_dow_day.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 2
  )


fig.update_layout(title= dict(text= "<b>Tippwahrscheinlichkeit nach Wochentagen</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), legend_title_text = "<b>Tage</b>", height= 780)
fig.update_xaxes(title_text="<b>Tage</b>", ticks= "outside", col= 1, row= 2)
fig.update_yaxes(title_text="<b>Tipp Wshk.</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 2, tickvals= [0.1, 0.3, 0.5])
fig.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))
fig.show()

- Die Trinkgeldwahrscheinlichkeit ist am Montagen 50.3% und Dienstagen 52.5% am höchstens

- Im Schnitt ist es damit ca. 14% wahrscheinlicher Trinkgeld an Montagen und Dienstagen zu erhalten, als an den restlichen Wochentagen


#### Attribut: order_number

- Gruppieren der Bestellungen nach order_number pro User und berechnen der Trinkgeldwahrscheinlichkeit für Intervalle der Bestellungsanzahl 

In [413]:
df_order_number1= df_p3.groupby(["user_id", "order_id", "order_number"], as_index= False).agg(tip= ("tip", "mean")).sort_values(by= ["user_id", "order_number"])

In [414]:
df_order_number2 = df_order_number1.groupby("user_id", as_index= False).agg(order_id_max = ("order_number", "max"), tip= ("tip", "sum"), orders= ("order_id", "nunique"))

In [415]:
df_order_number2 = df_order_number1.groupby("user_id", as_index= False).agg(order_id_max = ("order_number", "max"), tip= ("tip", "sum"), orders= ("order_id", "nunique"))

order_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
df_order_number2["order_number_cut"] = pd.cut(df_order_number2["order_id_max"], bins= order_bins, include_lowest= False)
df_order_number2.head()

,user_id,order_id_max,tip,orders,order_number_cut
0,3,12,10.0,12,"(10, 20]"
1,7,21,12.0,21,"(20, 30]"
2,14,14,8.0,14,"(10, 20]"
3,23,5,0.0,5,"(0, 10]"
4,25,3,2.0,3,"(0, 10]"


In [416]:
df_order_number3 = df_order_number2.groupby("order_number_cut", as_index= False).agg(users= ("user_id", "nunique"), order_max= ("order_id_max", "nunique"), tip= ("tip", "sum"), orders= ("orders", "sum"))
df_order_number3["tip_norm"] = (df_order_number3.tip / df_order_number3.orders).round(3)
df_order_number3["orders_norm"] = (df_order_number3.orders / df_order_number3.orders.sum()).round(3)
df_order_number3["order_number_cut"] = df_order_number3.order_number_cut.astype("str")
df_order_number3.head()

,order_number_cut,users,order_max,tip,orders,tip_norm,orders_norm
0,"(0, 10]",19369,8,32250.0,118378,0.272,0.195
1,"(10, 20]",8959,10,47207.0,131584,0.359,0.217
2,"(20, 30]",3859,10,43187.0,95863,0.451,0.158
3,"(30, 40]",2011,10,36084.0,70337,0.513,0.116
4,"(40, 50]",1249,10,30464.0,56223,0.542,0.093


**Plotten Trinkgeldwahrscheinlichkeit nach Bestellungsanzahl pro User**

In [417]:
# Plot P3: order_number 

fig = make_subplots(cols= 1, rows=2, specs= [[dict(type= "table")], [dict(type= "bar")]], vertical_spacing= 0.07)

fig.add_trace(
  go.Table(
    columnorder= [1, 2, 3, 4, 5, 6, 7],
    columnwidth= [1, 1, 1, 1, 1, 1, 1],
    header=dict(
        values= list(df_order_number3.columns),
        fill_color= px.colors.sequential.Purpor[5],
        line_color= px.colors.sequential.Greys[2],
        font= dict(color= "white", size=15),
        height= 35
        ),
    cells=dict(
        values= df_order_number3.T.values,
        fill_color= "white",
        line_color= px.colors.sequential.Greys[2],
        font= dict(color= "black", size=13),
        align= "center",
        height= 30
        )
    ),
    col= 1,
    row= 1
)


for interval in df_order_number3.order_number_cut.unique():

 if 0.5 <= df_order_number3.loc[df_order_number3.order_number_cut == interval].tip_norm.values:
  color = px.colors.sequential.Purpor[5]

  fig.add_trace(
   go.Bar(
    x= df_order_number3.loc[df_order_number3.order_number_cut == interval].order_number_cut.values,
    y= df_order_number3.loc[df_order_number3.order_number_cut == interval].tip_norm.values,
    name= f"Tipp: {(df_order_number3.loc[df_order_number3.order_number_cut == interval].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{interval}",
    legendgrouptitle_text = f"{interval}",
    hovertemplate = f"<b>{interval}</b><br><br>" +
        "Tipp: %{y:.0%}<br>" +
        "<extra></extra>",
   ),
  )

 else:
  color = px.colors.sequential.Greys[3]

  fig.add_trace(
   go.Bar(
    x= df_order_number3.loc[df_order_number3.order_number_cut == interval].order_number_cut.values,
    y= df_order_number3.loc[df_order_number3.order_number_cut == interval].tip_norm.values,
    name= f"Tipp: {(df_order_number3.loc[df_order_number3.order_number_cut == interval].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{interval}",
    legendgrouptitle_text = f"{interval}",
    hovertemplate = f"<b>{interval}</b><br><br>" +
        "Tipp: %{y:.0%}<br>" +
        "<extra></extra>",
   ),
  )

fig.update_layout(title= dict(text= "<b>Tippwahrscheinlichkeit nach Bestellungsanzahl pro User</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), legend_title_text = "<b>Bestellungen</b>", height= 800)
fig.update_xaxes(title_text="<b>Bestellungsanzahl in Intervallen</b>", ticks= "outside", col= 1, row= 2, showline= True)
fig.update_yaxes(title_text="<b>Tipp Wshk.</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 2, showline= True, tickvals= [0, 0.25, 0.5, 0.75])
fig.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))

fig.show()

- Bestellungen von Usern mit einer Bestellungsanzahl von 40 - 50 Bestellungen haben mit rund 54% die höchste Trinkgeldwahrscheinlichkeit

#### Attribut: order_hour_of_day

- Gruppieren der Bestelldaten nach Tagesstunden und berechnen der Trinkgeldwahrscheinlichkeit 

In [418]:
df_h = df_p3.groupby(["order_hour_of_day", "order_id"], as_index= False).agg(tip= ("tip", "mean"))
df_hour = df_h.groupby(["order_hour_of_day"], as_index= False).agg(orders= ("order_id", "nunique"), tip= ("tip", "sum"))
df_hour["tip_norm"] = df_hour.tip / df_hour.orders
df_hour["orders_norm"] = df_hour.orders / df_hour.orders.sum()
df_hour.head()

,order_hour_of_day,orders,tip,tip_norm,orders_norm
0,0,4072,2079.0,0.510560,0.006709
1,1,2219,1125.0,0.506985,0.003656
2,2,1340,694.0,0.517910,0.002208
3,3,956,477.0,0.498954,0.001575
4,4,986,510.0,0.517241,0.001625


**Plotten der Tippwahrscheinlichkeit pro Stunde**

In [419]:
# Plot P3: Order_hour_of_day

fig1 = make_subplots(cols= 1, rows= 2, shared_xaxes= True, shared_yaxes= False, vertical_spacing= 0)

for order_hour_of_day in df_hour.order_hour_of_day:

 if df_hour[df_hour.order_hour_of_day == order_hour_of_day].tip_norm.values >= 0.5:
  color = px.colors.sequential.Purpor[5]
  text= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm.values.round(3)

  fig1.add_trace(
   go.Bar(
    x= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day,
    y= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm,
    name= f"Tipp: {(df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{order_hour_of_day}",
    legendgrouptitle_text = f"{order_hour_of_day}.00 Uhr",
    hovertemplate = f"<b>{df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day.values[0]}.00 Uhr</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day,
    y= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm,
    name= f"Orders: {(df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm.values[0] * 100).round(3)}%",
    base = -df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{order_hour_of_day}",
    hovertemplate = f"<b>{df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day.values[0]}.00 Uhr</b><br><br>" +
                "Orders: %{base}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 2
  )

 elif 0.4 < df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm.values < 0.5:
  color= px.colors.sequential.Greys[3]

  fig1.add_trace(
   go.Bar(
    x= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day,
    y= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm,
    name= f"Tipp: {(df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{order_hour_of_day}",
    legendgrouptitle_text = f"{order_hour_of_day}.00 Uhr",
    hovertemplate = f"<b>{df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day.values[0]}.00 Uhr</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day,
    y= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm,
    name= f"Orders: {(df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm.values[0] * 100).round(3)}%",
    base = -df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{order_hour_of_day}",
    hovertemplate = f"<b>{df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day.values[0]}.00 Uhr</b><br><br>" +
                "Orders: %{base}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 2
  )

 elif 0.3 < df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm.values < 0.4:
  color= px.colors.sequential.Greys[2]

  fig1.add_trace(
   go.Bar(
    x= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day,
    y= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm,
    name= f"Tipp: {(df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{order_hour_of_day}",
    legendgrouptitle_text = f"{order_hour_of_day}.00 Uhr",
    hovertemplate = f"<b>{df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day.values[0]}.00 Uhr</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day,
    y= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm,
    name= f"Orders: {(df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm.values[0] * 100).round(3)}%",
    base = -df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{order_hour_of_day}",
    hovertemplate = f"<b>{df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day.values[0]}.00 Uhr</b><br><br>" +
                "Orders: %{base}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 2
  )

fig1.add_hline(
 y= 0,
 opacity= 0.3,
 line_dash= "solid",
 line_color= "grey",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= 0,
 opacity= 0.3,
 line_dash= "solid",
 line_color= "grey",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= -0.1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 2
)

fig1.update_annotations(opacity= 0.8)
fig1.update_layout(title= dict(text= "<b>Tippwahrscheinlichkeit nach Tagesstunden</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), legend_title_text = "<b>Tagesstunde</b>", height= 700)
fig1.update_xaxes(title_text="<b>Stunden</b>", ticks= "outside", col= 1, row= 2)
fig1.update_xaxes(showline= False, col= 1, row= 1)
fig1.update_yaxes(title_text="<b>Tipp Wshk.</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 1, showline= False)
fig1.update_yaxes(title_text="<b>Order Anteil</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 2, tickvals= [0, -0.025, -0.05, -0.075, -0.1])
fig1.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))
fig1.show()

In [420]:
# Plot P3: Order_hour_of_day
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
  go.Scatter(
    x= df_hour.order_hour_of_day,
    y= df_hour.orders_norm,
    mode= "lines+markers", 
    marker= dict(color= px.colors.sequential.Greys[4], size= 4),
    name= "Bestellungen",
    fill= "tozeroy",
    hoverinfo = "y"
  ),
  secondary_y= True
)


for hour in df_hour.order_hour_of_day:

 if df_hour[df_hour.order_hour_of_day == hour].tip_norm.values >= 0.5:
  color = px.colors.sequential.Purpor[5]
  text= df_hour.loc[df_hour.order_hour_of_day == hour].tip_norm.values.round(2)

  fig.add_trace(
   go.Bar(
    x= df_hour.loc[df_hour.order_hour_of_day == hour].order_hour_of_day,
    y= df_hour.loc[df_hour.order_hour_of_day == hour].tip_norm,
    name= f"Tipp: {(df_hour.loc[df_hour.order_hour_of_day == hour].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{hour}",
    legendgrouptitle_text= f"{int(hour)}",
    hovertemplate = f"<b>{df_hour.loc[df_hour.order_hour_of_day == hour].order_hour_of_day.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   secondary_y= False
  )

 elif 0.4 < df_hour.loc[df_hour.order_hour_of_day == hour].tip_norm.values < 0.5:
  color= px.colors.sequential.Greys[3]

  fig.add_trace(
   go.Bar(
    x= df_hour.loc[df_hour.order_hour_of_day == hour].order_hour_of_day,
    y= df_hour.loc[df_hour.order_hour_of_day == hour].tip_norm,
    name= f"Tipp: {(df_hour.loc[df_hour.order_hour_of_day == hour].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{hour}",
    legendgrouptitle_text= f"{int(hour)}",
    hovertemplate = f"<b>{df_hour.loc[df_hour.order_hour_of_day == hour].order_hour_of_day.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   secondary_y= False
  )

 elif 0.3 < df_hour.loc[df_hour.order_hour_of_day == hour].tip_norm.values < 0.4:
  color= px.colors.sequential.Greys[2]

  fig.add_trace(
   go.Bar(
    x= df_hour.loc[df_hour.order_hour_of_day == hour].order_hour_of_day,
    y= df_hour.loc[df_hour.order_hour_of_day == hour].tip_norm,
    name= f"Tipp: {(df_hour.loc[df_hour.order_hour_of_day == hour].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{hour}",
    legendgrouptitle_text= f"{int(hour)}",
    hovertemplate = f"<b>{df_hour.loc[df_hour.order_hour_of_day == hour].order_hour_of_day.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   secondary_y= False
  )

 elif 0.2 < df_hour.loc[df_hour.order_hour_of_day == hour].tip_norm.values < 0.3:
  color= px.colors.sequential.Greys[1]
  text= df_hour.loc[df_hour.order_hour_of_day == hour].tip_norm.values.round(2)

  fig.add_trace(
   go.Bar(
    x= df_hour.loc[df_hour.order_hour_of_day == hour].order_hour_of_day,
    y= df_hour.loc[df_hour.order_hour_of_day == hour].tip_norm,
    name= f"Tipp: {(df_hour.loc[df_hour.order_hour_of_day == hour].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{hour}",
    legendgrouptitle_text= f"{int(hour)}",
    hovertemplate = f"<b>{df_hour.loc[df_hour.order_hour_of_day == hour].order_hour_of_day.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   secondary_y= False
  )


fig.update_layout(title= dict(text= "<b>Tippwahrscheinlichkeit über den Tag verteilt</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), legend_title_text = "<b>Tage</b>", height= 600)
fig.update_xaxes(title_text="<b>Tage</b>", ticks= "outside")
fig.update_yaxes(title_text="<b>Tipp Wshk.</b>", ticks= "outside", showgrid= True, gridcolor= px.colors.sequential.Greys[1])
fig.update_yaxes(title_text="<b>Order Anteil</b>", ticks= "outside", showgrid= False, secondary_y= True, showline= True, linecolor= "black")
fig.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))
fig.show()

- Im Zeitraum von 5 bis 18 Uhr liegt die Trinkgeldwahrscheinlichtkeit ausnahmslos unterhalb von 50%
 
- Außerhalb der "normalen" Arbeitszeit, von 19 bis 4 Uhr, liegt die Trinkgeldwahrscheinlichkeit außer aum 4 Uhr immer über 50%
 
- Bestellungen die um 21 zugestellt wurden haben mit 54.6% die höchste Wahrscheinlichkeit Trinkgeld zu erhalten 

#### Attribut: days_since_prior_order

- Gruppieren der Bestelldaten nach days_since_prior_order und berechnen der Tippwahrscheinlichkeit

In [421]:
df_days = df_p3.groupby(["days_since_prior_order", "order_id"], as_index=False).agg(tip_mean= ("tip", "mean"))
df_days_since_prior_order= df_days.groupby("days_since_prior_order", as_index= False).agg(tip= ("tip_mean", "sum"), orders= ("order_id", "nunique"))
df_days_since_prior_order["tip_norm"] = df_days_since_prior_order.tip / df_days_since_prior_order.orders
df_days_since_prior_order["orders_norm"] = df_days_since_prior_order.orders / df_days_since_prior_order.orders.sum()
df_days_since_prior_order.head()

,days_since_prior_order,tip,orders,tip_norm,orders_norm
0,0.0,5159.0,12005,0.429738,0.021078
1,1.0,10188.0,26178,0.389182,0.045963
2,2.0,14490.0,34983,0.414201,0.061422
3,3.0,17048.0,38541,0.442334,0.067669
4,4.0,18052.0,39479,0.457256,0.069316


**Plotten der Trinkgeldwahrscheinlichkeit anhand wie viele Tage die vorherige Bestellungen zurückliegt**

In [422]:
# Plot P3: Days_since_prior_order
fig1 = make_subplots(cols= 1, rows= 2, shared_xaxes= True, shared_yaxes= False, vertical_spacing= 0)

for days_since_prior_order in df_days_since_prior_order.days_since_prior_order:

 if df_days_since_prior_order[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values >= 0.5:
  color = px.colors.sequential.Purpor[5]
  text= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values.round(2)

  fig1.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm,
    name= f"{int(days_since_prior_order)}: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{days_since_prior_order}",
    hovertemplate = f"<b>{df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm,
    name= f"Orders: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm.values[0] * 100).round(3)}%",
    base = -df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{days_since_prior_order}",
    hovertemplate = f"<b>{df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 2
  )

 elif 0.4 < df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values < 0.5:
  color= px.colors.sequential.Greys[3]

  fig1.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm,
    name= f"{int(days_since_prior_order)}: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{days_since_prior_order}",
    hovertemplate = f"<b>{df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm,
    name= f"Orders: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm.values[0] * 100).round(3)}%",
    base = -df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{days_since_prior_order}",
    hovertemplate = f"<b>{df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 2
  )

 elif 0.3 < df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values < 0.4:
  color= px.colors.sequential.Greys[2]

  fig1.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm,
    name= f"{int(days_since_prior_order)}: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{days_since_prior_order}",
    hovertemplate = f"<b>{df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm,
    name= f"Orders: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm.values[0] * 100).round(3)}%",
    base = -df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{days_since_prior_order}",
    hovertemplate = f"<b>{df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 2
  )

 elif 0.2 < df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values < 0.3:
  color= px.colors.sequential.Greys[1]
  text= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values.round(2)

  fig1.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm,
    name= f"{int(days_since_prior_order)}: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{days_since_prior_order}",
    hovertemplate = f"<b>{df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm,
    name= f"Orders: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm.values[0] * 100).round(3)}%",
    base = -df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{days_since_prior_order}",
    hovertemplate = f"<b>{df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 2
  )


fig1.add_hline(
 y= 0,
 opacity= 0.3,
 line_dash= "solid",
 line_color= "grey",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= 0,
 opacity= 0.3,
 line_dash= "solid",
 line_color= "grey",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -0.5,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -0.5,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= -0.18,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 2
)

fig1.update_annotations(opacity= 0.8)
fig1.update_layout(title= dict(text= "<b>Tippwahrscheinlichkeit von Bestellungen mit Tagen seit der letzten Bestellungen</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), legend_title_text = "<b>Tage</b>", height= 700)
fig1.update_xaxes(title_text="<b>Tage</b>", ticks= "outside", col= 1, row= 2)
fig1.update_xaxes(showline= False, col= 1, row= 1)
fig1.update_yaxes(title_text="<b>Tipp Wshk.</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 1, showline= False)
fig1.update_yaxes(title_text="<b>Order Anteil</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 2)
fig1.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))
fig1.show()

In [423]:
# Plot P3: Days_since_prior_order
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
  go.Scatter(
    x= df_days_since_prior_order.days_since_prior_order,
    y= df_days_since_prior_order.orders_norm,
    mode= "lines+markers", 
    marker= dict(color= px.colors.sequential.Greys[4], size= 4),
    name= "Bestellungen",
    fill= "tozeroy",
    hoverinfo = "y",
  ),
  secondary_y= True
)


for days_since_prior_order in df_days_since_prior_order.days_since_prior_order:

 if df_days_since_prior_order[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values >= 0.5:
  color = px.colors.sequential.Purpor[5]
  text= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values.round(2)

  fig.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm,
    name= f"Tipp: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{days_since_prior_order}",
    legendgrouptitle_text= f"{int(days_since_prior_order)}",
    hovertemplate = f"<b>{df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   secondary_y= False
  )

 elif 0.4 < df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values < 0.5:
  color= px.colors.sequential.Greys[3]

  fig.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm,
    name= f"Tipp: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{days_since_prior_order}",
    legendgrouptitle_text= f"{int(days_since_prior_order)}",
    hovertemplate = f"<b>{df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   secondary_y= False
  )

 elif 0.3 < df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values < 0.4:
  color= px.colors.sequential.Greys[2]

  fig.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm,
    name= f"Tipp: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{days_since_prior_order}",
    legendgrouptitle_text= f"{int(days_since_prior_order)}",
    hovertemplate = f"<b>{df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   secondary_y= False
  )

 elif 0.2 < df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values < 0.3:
  color= px.colors.sequential.Greys[1]
  text= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values.round(2)

  fig.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm,
    name= f"Tipp: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{days_since_prior_order}",
    legendgrouptitle_text= f"{int(days_since_prior_order)}",
    hovertemplate = f"<b>{df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   secondary_y= False
  )


fig.update_layout(title= dict(text= "<b>Tippwahrscheinlichkeit von Bestellungen mit Tagen seit der letzten Bestellungen</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), legend_title_text = "<b>Tage</b>", height= 600)
fig.update_xaxes(title_text="<b>Tage</b>", ticks= "outside")
fig.update_yaxes(title_text="<b>Tipp Wshk.</b>", ticks= "outside", showgrid= True, gridcolor= px.colors.sequential.Greys[1])
fig.update_yaxes(title_text="<b>Order Anteil</b>", ticks= "outside", showgrid= False, secondary_y= True, showline= True, linecolor= "black")
fig.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))
fig.show()

- Bestellungen die 7 Tage nach der vorherigen erfolgen haben die höchste Trinkgeldwahrscheinlichkeit

- Die Tippwahrscheinlichtkeit sinkt je mit der Anzahl an Tagen die zwischen den Bestellungen liegen, steigt jedoch periodisch alle 7 Tage wieder leicht an 

#### Attribut: department

- Gruppieren der Produkte jeder Bestellung nachedem aus welchem department sie stammen   

In [424]:
df_dep = df_p3.groupby(["department", "order_id"], as_index= False).agg(tip_mean= ("tip", "mean"), all= ("order_id", "count"))
df_dep1 = df_dep.groupby("department", as_index= False).agg(tip = ("tip_mean", "sum"), orders= ("order_id", "nunique"))
df_dep1["tip_norm"] = df_dep1.tip / df_dep1.orders
df_dep1["orders_norm"] = df_dep1.orders / df_dep1.orders.sum()
df_dep2 = df_dep1.sort_values(by= "tip_norm", ascending= False)

In [425]:
df_dep2.head()

,department,tip,orders,tip_norm,orders_norm
0,alcohol,12531.0,15569,0.804869,0.005399
4,breakfast,53783.0,100391,0.535735,0.034813
1,babies,17786.0,33891,0.524800,0.011753
5,bulk,3448.0,6618,0.521003,0.002295
20,snacks,133047.0,263766,0.504413,0.091468


**Plotten der Trinkgeldwahrscheinlichkeit der Departments**

In [426]:
# Plot P3: Department

fig1 = make_subplots(cols= 1, rows= 2, shared_xaxes= True, shared_yaxes= False, vertical_spacing= 0)

for department in df_dep2.department:

 if df_dep2[df_dep2.department == department].tip_norm.values >= 0.5:
  color = px.colors.sequential.Purpor[5]
  text= df_dep2.loc[df_dep2.department == department].tip_norm.values.round(3)

  fig1.add_trace(
   go.Bar(
    x= df_dep2.loc[df_dep2.department == department].department,
    y= df_dep2.loc[df_dep2.department == department].tip_norm,
    name= f"Tipp: {(df_dep2.loc[df_dep2.department == department].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{department}",
    legendgrouptitle_text = f"{department}",
    hovertemplate = f"<b>{df_dep2.loc[df_dep2.department == department].department.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_dep2.loc[df_dep2.department == department].department,
    y= df_dep2.loc[df_dep2.department == department].orders_norm,
    name= f"Orders: {(df_dep2.loc[df_dep2.department == department].orders_norm.values[0] * 100).round(3)}%",
    base = -df_dep2.loc[df_dep2.department == department].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{department}",
    legendgrouptitle_text = f"{department}",
    hovertemplate = f"<b>{df_dep2.loc[df_dep2.department == department].department.values[0]}</b><br><br>" +
                "Orders: %{base}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 2
  )

 elif 0.4 < df_dep2.loc[df_dep2.department == department].tip_norm.values < 0.5:
  color= px.colors.sequential.Greys[3]

  fig1.add_trace(
   go.Bar(
    x= df_dep2.loc[df_dep2.department == department].department,
    y= df_dep2.loc[df_dep2.department == department].tip_norm,
    name= f"Tipp: {(df_dep2.loc[df_dep2.department == department].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{department}",
    legendgrouptitle_text = f"{department}",
    hovertemplate = f"<b>{df_dep2.loc[df_dep2.department == department].department.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_dep2.loc[df_dep2.department == department].department,
    y= df_dep2.loc[df_dep2.department == department].orders_norm,
    name= f"Orders: {(df_dep2.loc[df_dep2.department == department].orders_norm.values[0] * 100).round(3)}%",
    base = -df_dep2.loc[df_dep2.department == department].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{department}",
    legendgrouptitle_text = f"{department}",
    hovertemplate = f"<b>{df_dep2.loc[df_dep2.department == department].department.values[0]}</b><br><br>" +
                "Orders: %{base}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 2
  )

 elif 0.3 < df_dep2.loc[df_dep2.department == department].tip_norm.values < 0.4:
  color= px.colors.sequential.Greys[2]

  fig1.add_trace(
   go.Bar(
    x= df_dep2.loc[df_dep2.department == department].department,
    y= df_dep2.loc[df_dep2.department == department].tip_norm,
    name= f"Tipp: {(df_dep2.loc[df_dep2.department == department].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{department}",
    legendgrouptitle_text = f"{department}",
    hovertemplate = f"<b>{df_dep2.loc[df_dep2.department == department].department.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_dep2.loc[df_dep2.department == department].department,
    y= df_dep2.loc[df_dep2.department == department].orders_norm,
    name= f"Orders: {(df_dep2.loc[df_dep2.department == department].orders_norm.values[0] * 100).round(3)}%",
    base = -df_dep2.loc[df_dep2.department == department].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{department}",
    legendgrouptitle_text = f"{department}",
    hovertemplate = f"<b>{df_dep2.loc[df_dep2.department == department].department.values[0]}</b><br><br>" +
                "Orders: %{base}<br>" +
                "<extra></extra>"
   ),
   col= 1,
   row= 2
  )

fig1.add_hline(
 y= 0,
 opacity= 0.3,
 line_dash= "solid",
 line_color= "grey",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= 0,
 opacity= 0.3,
 line_dash= "solid",
 line_color= "grey",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= -0.18,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 2
)

fig1.update_annotations(opacity= 0.8)
fig1.update_layout(title= dict(text= "<b>Tippwahrscheinlichkeit der Departments</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), legend_title_text = "<b>Departments</b>", height= 700)
fig1.update_xaxes(title_text="<b>Departments</b>", ticks= "outside", col= 1, row= 2)
fig1.update_xaxes(showline= False, col= 1, row= 1)
fig1.update_yaxes(title_text="<b>Tipp Wshk.</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 1, showline= False)
fig1.update_yaxes(title_text="<b>Order Anteil</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 2)
fig1.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))
fig1.show()

In [427]:
# Plot P3: Department
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
  go.Scatter(
    x= df_dep2.department,
    y= df_dep2.orders_norm,
    mode= "lines+markers", 
    marker= dict(color= px.colors.sequential.Greys[4], size= 4),
    name= "Bestellungen",
    fill= "tozeroy",
    hoverinfo = "y",
  ),
  secondary_y= True
)


for department in df_dep2.department:

 if df_dep2[df_dep2.department == department].tip_norm.values >= 0.5:
  color = px.colors.sequential.Purpor[5]
  text= df_dep2.loc[df_dep2.department == department].tip_norm.values.round(2)

  fig.add_trace(
   go.Bar(
    x= df_dep2.loc[df_dep2.department == department].department,
    y= df_dep2.loc[df_dep2.department == department].tip_norm,
    name= f"Tipp: {(df_dep2.loc[df_dep2.department == department].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{department}",
    legendgrouptitle_text= f"{department}",
    hovertemplate = f"<b>{df_dep2.loc[df_dep2.department == department].department.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   secondary_y= False
  )

 elif 0.4 < df_dep2.loc[df_dep2.department == department].tip_norm.values < 0.5:
  color= px.colors.sequential.Greys[3]

  fig.add_trace(
   go.Bar(
    x= df_dep2.loc[df_dep2.department == department].department,
    y= df_dep2.loc[df_dep2.department == department].tip_norm,
    name= f"Tipp: {(df_dep2.loc[df_dep2.department == department].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{department}",
    legendgrouptitle_text= f"{department}",
    hovertemplate = f"<b>{df_dep2.loc[df_dep2.department == department].department.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   secondary_y= False
  )

 elif 0.3 < df_dep2.loc[df_dep2.department == department].tip_norm.values < 0.4:
  color= px.colors.sequential.Greys[2]

  fig.add_trace(
   go.Bar(
    x= df_dep2.loc[df_dep2.department == department].department,
    y= df_dep2.loc[df_dep2.department == department].tip_norm,
    name= f"Tipp: {(df_dep2.loc[df_dep2.department == department].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{department}",
    legendgrouptitle_text= f"{department}",
    hovertemplate = f"<b>{df_dep2.loc[df_dep2.department == department].department.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   secondary_y= False
  )

 elif 0.2 < df_dep2.loc[df_dep2.department == department].tip_norm.values < 0.3:
  color= px.colors.sequential.Greys[1]
  text= df_dep2.loc[df_dep2.department == department].tip_norm.values.round(2)

  fig.add_trace(
   go.Bar(
    x= df_dep2.loc[df_dep2.department == department].department,
    y= df_dep2.loc[df_dep2.department == department].tip_norm,
    name= f"Tipp: {(df_dep2.loc[df_dep2.department == department].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{department}",
    legendgrouptitle_text= f"{department}",
    hovertemplate = f"<b>{df_dep2.loc[df_dep2.department == department].department.values[0]}</b><br><br>" +
                "Tipp: %{y}<br>" +
                "<extra></extra>"
   ),
   secondary_y= False
  )


fig.update_layout(title= dict(text= "<b>Tippwahrscheinlichkeit der Departments</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), legend_title_text = "<b>Tage</b>", height= 600)
fig.update_xaxes(title_text="<b>Tage</b>", ticks= "outside")
fig.update_yaxes(title_text="<b>Tipp Wshk.</b>", ticks= "outside", showgrid= True, gridcolor= px.colors.sequential.Greys[1])
fig.update_yaxes(title_text="<b>Order Anteil</b>", ticks= "outside", showgrid= False, secondary_y= True, showline= True, linecolor= "black")
fig.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))
fig.show()

- Bestellungen die Produkte aus den Departments alcohol, breakfast, babies, bulk und snacks enthalten haben eine Trinkgeldwahrscheinlichkeit > 50%

#### Attribut: add_to_cart_order

- Gruppieren der Bestellungen nach ihrer Bestellgröße und anschließend unterteilen in Intervalle

In [428]:
df_1 = df.groupby(["order_id"], as_index= False).agg(order_size= ("add_to_cart_order", "max"), tip= ("tip", "mean"))
df_2 = df_1.groupby(["order_size"], as_index= False).agg(tip= ("tip", "sum"), orders= ("order_id", "nunique"))
df_2["tip_norm"] = df_2.tip / df_2.orders
df_2["orders_norm"] = df_2.orders / df_2.orders.sum()
df_2.head()

,order_size,tip,orders,tip_norm,orders_norm
0,1,9972.0,29507,0.337954,0.048618
1,2,12611.0,34892,0.361430,0.057491
2,3,14360.0,38512,0.372871,0.063455
3,4,16013.0,41678,0.384207,0.068672
4,5,16877.0,43199,0.390680,0.071178


In [429]:
ordersize_bins = [1, 5, 10, 15, 20, 25, 30, 35, 150]
df_2["order_size_cut"] = pd.cut(df_2["order_size"], bins= ordersize_bins, include_lowest= True)

In [430]:
df_2_cut = df_2.groupby(["order_size_cut"], as_index= False).agg(tip= ("tip", "sum"), tip_norm= ("tip_norm", "mean"), orders= ("orders", "sum"),)
df_2_cut["orders_norm"] = (df_2_cut.orders / df_2_cut.orders.sum()).round(5)
df_2_cut["tip_norm"] = df_2_cut.tip_norm.round(3)
df_2_cut["order_size_cut"] = df_2_cut.order_size_cut.astype("str")
df_2_cut

,order_size_cut,tip,tip_norm,orders,orders_norm
0,"(0.999, 5.0]",69833.0,0.369,187788,0.30941
1,"(5.0, 10.0]",77444.0,0.409,189677,0.31253
2,"(10.0, 15.0]",49136.0,0.442,111613,0.18390
3,"(15.0, 20.0]",28496.0,0.472,60484,0.09966
4,"(20.0, 25.0]",14876.0,0.489,30443,0.05016
5,"(25.0, 30.0]",7380.0,0.502,14747,0.02430
6,"(30.0, 35.0]",3319.0,0.500,6660,0.01097
7,"(35.0, 150.0]",2893.0,0.634,5503,0.00907


**Plotten der Trinkgeldwahrscheinlichkeit anhand der Bestellgröße**

In [431]:
# Plot P3: Order_size

fig = make_subplots(cols= 1, rows=2, specs= [[dict(type= "table")], [dict(type= "bar")]], vertical_spacing= 0.03)

fig.add_trace(
  go.Table(
    columnorder= [1, 2, 3, 4, 5],
    columnwidth= [1, 1, 1, 1, 1],
    header=dict(
        values= list(df_2_cut.columns),
        fill_color= px.colors.sequential.Purpor[5],
        line_color= px.colors.sequential.Greys[2],
        font= dict(color= "white", size=15),
        height= 35
        ),
    cells=dict(
        values= df_2_cut.T.values,
        fill_color= "white",
        line_color= px.colors.sequential.Greys[2],
        font= dict(color= "black", size=13),
        align= "center",
        height= 30
        )
    ),
    col= 1,
    row= 1
)


for interval in df_2_cut.order_size_cut.unique():

 if df_2_cut.loc[df_2_cut.order_size_cut == interval].tip_norm.values >= 0.5:
  color = px.colors.sequential.Purpor[5]
  text= df_2_cut.loc[df_2_cut.order_size_cut == interval].tip_norm.values

  fig.add_trace(
   go.Bar(
    x= df_2_cut.loc[df_2_cut.order_size_cut == interval].order_size_cut.values,
    y= df_2_cut.loc[df_2_cut.order_size_cut == interval].tip_norm.values,
    name= f"Tipp: {(df_2_cut.loc[df_2_cut.order_size_cut == interval].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text = text,
    legendgroup= f"{interval}",
    legendgrouptitle_text = f"{interval}",
    hovertemplate = f"<b>{interval}</b><br><br>" +
        "Tipp: %{y:.0%}<br>" +
        "<extra></extra>",
   ),
  )

 else:
  color = px.colors.sequential.Greys[3]

  fig.add_trace(
   go.Bar(
    x= df_2_cut.loc[df_2_cut.order_size_cut == interval].order_size_cut.values,
    y= df_2_cut.loc[df_2_cut.order_size_cut == interval].tip_norm.values,
    name= f"Tipp: {(df_2_cut.loc[df_2_cut.order_size_cut == interval].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{interval}",
    legendgrouptitle_text = f"{interval}",
    hovertemplate = f"<b>{interval}</b><br><br>" +
        "Tipp: %{y:.0%}<br>" +
        "<extra></extra>",
   ),
  )


fig.update_layout(title= dict(text= "<b>Tippwahrscheinlichkeit jeder Bestellungsgröße</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), legend_title_text = "<b>Bestellgröße</b>", height= 800)
fig.update_xaxes(title_text="<b>Bestellgröße in Intervallen</b>", ticks= "outside", col= 1, row= 2, showline= True)
fig.update_yaxes(title_text="<b>Tipp Wshk.</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 2, showline= True, tickvals= [0, 0.25, 0.5, 0.75])
fig.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))

fig.show()

- Bestellungen mit ab einer Bestellgröße von >25 haben eine Trinkgeldwahrscheinlichkeit von >= 50%

- Das Maximum der Trinkgeldwahrscheinlichkeit liegt bei  Bestellungen mit einer Bestellgröße von 35 bis 150 Produkten, dabei sei gesagt, dass Intervall größer ist, da in diesem sich nur 0.1% der Bestelldaten befinden

### **P4:** Spielt die Vergangenheit eines Users eine Rolle in Hinblick auf die Trinkgeldwahrscheinlichkeit, oder kommt es nur auf Inhalt und Parameter der aktuellen Bestellung an?

- Gruppieren der Bestellungen nach User und identifizierend der aktuellen Bestellung pro User

- User Trinkgeldwahrscheinlichkeit aus ihrer Bestellhistorie berechnen und mit der Trinkgeldgabe ihrer aktuellen Bestellung vergleichen 

##### Preprocessing

In [432]:
df_user_hist = df.copy()

In [433]:
df_user_hist = df_user_hist.sort_values(by= ["user_id", "order_number"], ascending= True)
df_user = df_user_hist.groupby("user_id", as_index= False).agg(orders = ("order_id", "unique"), reordered= ("reordered", "mean"), tip= ("tip", "mean"), order_number= ("order_number", "unique"))
df_user.head()

,user_id,orders,reordered,tip,order_number
0,3,"[244029, 78693, 533119, 361761, 481466, 350475...",0.625000,0.840909,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
1,7,"[455863, 426645, 21397, 301146, 589317, 298684...",0.679070,0.637209,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,14,"[515987, 569030, 242632, 245955, 300451, 37770...",0.343891,0.506787,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]"
3,23,"[318279, 1415, 337923, 190237, 603547]",0.192308,0.000000,"[1, 2, 3, 4, 5]"
4,25,"[463866, 531613, 45667]",0.095238,0.476190,"[1, 2, 3]"


In [434]:
last_orders = []

for ind, order in enumerate(df_user.orders):
 last_orders.append(df_user.orders[ind][-1])

df_user_hist.drop(df_user_hist.loc[df_user_hist.order_id.isin(last_orders)].index, inplace= True)

In [435]:
df_userh = df_user_hist.groupby("user_id", as_index= False).agg(orders = ("order_id", "nunique"), reordered= ("reordered", "mean"), tip= ("tip", "mean"))
df_userh.head()

,user_id,orders,reordered,tip
0,3,11,0.597561,0.829268
1,7,20,0.669903,0.621359
2,14,13,0.323810,0.533333
3,23,4,0.100000,0.000000
4,25,2,0.066667,0.266667


In [436]:
df_last_order = df.loc[df.order_id.isin(last_orders)].groupby("user_id").agg(tip_last_order= ("tip", "max"), last_order_id = ("order_id", "max")).reset_index()
df_last_order.head()

,user_id,tip_last_order,last_order_id
0,3,1,248921
1,7,1,93090
2,14,0,411382
3,23,0,603547
4,25,1,45667


In [437]:
df_userh = df_userh.merge(df_last_order, how= "right", on= "user_id")
df_userh.head()

,user_id,orders,reordered,tip,tip_last_order,last_order_id
0,3,11,0.597561,0.829268,1,248921
1,7,20,0.669903,0.621359,1,93090
2,14,13,0.323810,0.533333,0,411382
3,23,4,0.100000,0.000000,0,603547
4,25,2,0.066667,0.266667,1,45667


##### Analyse

**Bestellungen bei welcher der user eine tip_wshk von >= 0.5 hat verglichen mit seiner letzten/aktuellen Bestellung**


Unter der Annahme, dass wenn die tip_wshk eines users in der historie >= 0.5 ist, tendiert er eher dazu zu tippen bei seiner letzten/aktuellen Bestellung

**trifft zu:**

In [438]:
df_userh.loc[(df_userh.tip >= 0.5) & (df_userh.tip_last_order == 1)].head()

,user_id,orders,reordered,tip,tip_last_order,last_order_id
0,3,11,0.597561,0.829268,1,248921
1,7,20,0.669903,0.621359,1,93090
6,39,6,0.571429,0.500000,1,489351
13,73,6,0.363636,0.636364,1,443957
15,85,6,0.384615,0.807692,1,431764


In [439]:
case1_true = 7685 / 37367
case1_true

0.2056627505553028

**trifft nicht zu:**

In [440]:
df_userh.loc[(df_userh.tip >= 0.5) & (df_userh.tip_last_order == 0)].head() 

,user_id,orders,reordered,tip,tip_last_order,last_order_id
2,14,13,0.323810,0.533333,0,411382
20,110,20,0.556196,0.700288,0,12722
26,138,32,0.628378,0.533784,0,80767
27,140,76,0.735882,0.537793,0,454907
33,169,6,0.105882,0.917647,0,136338


In [441]:
case1_false = 4181 / 37367
case1_false

0.11189017047127145

**Bestellungen bei welcher der user eine tip_wshk von < 0.5 hat verglichen mit seiner letzten/aktuellen Bestellung**

Unter der Annahme, dass wenn die tip_wshk eines users in der historie < 0.5 ist, tendiert er eher dazu nicht zu tippen bei seiner letzten/aktuellen Bestellung

**trifft zu:**

In [442]:
df_userh.loc[(df_userh.tip < 0.5) & (df_userh.tip_last_order == 0)].head()

,user_id,orders,reordered,tip,tip_last_order,last_order_id
3,23,4,0.100000,0.000000,0,603547
9,53,3,0.028571,0.000000,0,548094
10,55,7,0.505747,0.000000,0,6945
11,65,14,0.393939,0.000000,0,190242
14,83,6,0.333333,0.179487,0,430576


In [443]:
case2_true = 20635 / 37367
case2_true

0.5522252254663206

**trifft nicht zu:**

In [444]:
df_userh.loc[(df_userh.tip < 0.5) & (df_userh.tip_last_order == 1)].head() 

,user_id,orders,reordered,tip,tip_last_order,last_order_id
4,25,2,0.066667,0.266667,1,45667
5,28,23,0.416667,0.303571,1,351338
7,40,8,0.641304,0.456522,1,203217
8,42,16,0.548872,0.127820,1,497941
12,72,12,0.680851,0.212766,1,463486


In [445]:
case2_false = 4866 / 37367
case2_false

0.1302218535071052

In [446]:
print(f"Wahrscheinlichkeit Trifft zu: {case1_true + case2_true}, Wahrscheinlichlkeit Trifft nicht zu: {case1_false + case2_false}")

Wahrscheinlichkeit Trifft zu: 0.7578879760216234, Wahrscheinlichlkeit Trifft nicht zu: 0.24211202397837664


##### Preprocessing und Feature Importance 

- Um zu belegen das die Vergangenheit eines Users die Trinkgeldwahrscheinlichkeit seiner aktuellen Bestellung beeinlfusst wird der Dataframe in zwei Datensätze unterteilt.   

- Im Ersten sind alle Bestellung der Historie eines Users und im Zweiten befinden sich nur die aktuellen Bestellungen eines Users.  

- Das Feature user_tip_ratio wird erstellt, welches die Tippwahrscheinlichkeit jedes Users bis Zeitpunkt seiner aktuellen Bestellung angibt.    

- Auf dem ersten Datensatz wird nun ein Entscheidungsbaum als Classifier trainiert der die Wahrscheinlichkeit, dass eine Bestellung einen Tipp erhält, (tip: 0/1) auf dem zweiten Datensatz (die aktuellen Bestellungen) versucht hervorzusagen. 

**Bestimmen der letzten/aktuellen Bestellung eines Users**

In [448]:
df_userhh = df.copy()

df_userhh1 = df_userhh.sample(frac= 0.1)

df_userhh2 = df_userhh1.sort_values(by= ["user_id", "order_number"], ascending= True)
df_userh1 = df_userhh2.groupby("user_id", as_index= False).agg(orders = ("order_id", "unique"), tip= ("tip", "mean"), order_number= ("order_number", "unique"))

last_orders = []
for ind, order in enumerate(df_userh1.orders):
 last_orders.append(df_userh1.orders[ind][-1])

**Aufteilen des Dataframes in Bestellungen aus der Vergangenheit und aktuelle Bestellungen**

In [449]:
df1 = df_userhh1.copy()

In [450]:
df_last_orders = df1.loc[df1.order_id.isin(last_orders)]

In [451]:
df1.drop(df1.loc[df1.order_id.isin(last_orders)].index, inplace= True)
df_tip = df1.groupby(["user_id"], as_index= False).agg(user_tip_ratio = ("tip", "mean"))
df_tip.head()

,user_id,user_tip_ratio
0,3,0.857143
1,7,0.600000
2,14,0.578947
3,23,0.000000
4,25,0.000000


**Zusammenstellen des Trainings und Testdatensatzes**

In [452]:
df1 = df1.merge(df_tip, on= "user_id", how= "right")
df_last_orders = df_last_orders.merge(df_tip, on= "user_id", how= "right")

**Trainingsdatensatz (Bestellungen vor der aktuelle Bestellung des Users):**

In [453]:
df1

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,user_tip_ratio
0,572446,22035,2,1,3,8,0,17,7.0,0,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Madera,0.857143
1,326696,1819,2,1,3,7,0,15,7.0,1,All Natural No Stir Creamy Almond Butter,88,13,pantry,spreads,Madera,0.857143
2,244029,24810,9,0,3,1,1,14,NaN,1,Organic Lightly Salted Brown Rice Cakes,78,19,snacks,crackers,Madera,0.857143
3,78693,21137,7,0,3,2,3,19,9.0,1,Organic Strawberries,24,4,produce,fresh fruits,Madera,0.857143
4,350475,39190,2,1,3,6,0,16,7.0,1,Vanilla Unsweetened Almond Milk,91,16,dairy eggs,soy lactosefree,Madera,0.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551802,361641,2180,20,1,206201,10,0,13,6.0,1,Angel Hair Pasta,131,9,dry goods pasta,dry pasta,Orange,0.611111
551803,575966,21267,1,1,206201,16,5,15,9.0,1,Sourdough Bread,112,3,bakery,bread,Orange,0.611111
551804,361641,30250,9,0,206201,10,0,13,6.0,1,American Grana Extra Aged Parmesan Cheese,2,16,dairy eggs,specialty cheeses,Orange,0.611111
551805,554910,29487,1,1,206201,4,4,3,13.0,0,Roma Tomato,83,4,produce,fresh vegetables,Orange,0.611111


**Testdatensatz (aktuelle Bestellungen eines Users):**

In [454]:
df_last_orders

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,user_tip_ratio
0,560872,47766,2,1,3,9,0,16,7.0,1,Organic Avocado,24,4,produce,fresh fruits,Madera,0.857143
1,93090,47272,2,1,7,21,2,11,6.0,1,Antioxidant Infusions Beverage Malawi Mango,31,7,beverages,refrigerated,Kern,0.600000
2,93090,37999,3,1,7,21,2,11,6.0,1,Antioxidant Infusions Ipanema Pomegranate Beve...,31,7,beverages,refrigerated,Kern,0.600000
3,602109,29509,2,1,14,13,5,5,7.0,1,80 Vodka Holiday Edition,124,5,alcohol,spirits,Nevada,0.578947
4,603547,32327,11,0,23,5,1,12,30.0,0,3 Gallon Food Scrap Bag,60,17,household,trash bags liners,Fresno,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58037,345993,18205,4,1,206189,6,0,22,4.0,1,Peanut Butter Gluten Free Breakfast Bars,48,14,breakfast,breakfast bars pastries,Yuba,0.250000
58038,569445,39719,5,1,206192,13,0,21,30.0,1,"Freshly Made. Filled with Creamy Ricotta, Aged...",12,9,dry goods pasta,fresh pasta,Ventura,0.000000
58039,118783,6475,2,1,206195,18,2,15,24.0,0,Limonata Sparkling Lemon Beverage,77,7,beverages,soft drinks,Sutter,0.000000
58040,118783,19982,1,0,206195,18,2,15,24.0,0,Part Skim Ricotta,108,16,dairy eggs,other creams cheeses,Sutter,0.000000


**One Hot Encoding der Spalte days_since_prior_order, da NAN-Values**

- müssen geändert werden da die Methode DeecisionTreeClassifier nicht mit NAN-Werten umgehen kann

In [455]:
df_train = pd.get_dummies(df1, columns= ["county", "days_since_prior_order"], dummy_na= True)

In [456]:
df_test = pd.get_dummies(df_last_orders, columns= ["county", "days_since_prior_order"], dummy_na= True)

##### Entscheidungsbaum und Feature Importance

- Trainings- und Testdatensplit

In [457]:
df_train.drop(["product_name", "department", "aisle"], axis= 1, inplace= True)
df_test.drop(["product_name", "department", "aisle"], axis= 1, inplace= True)

In [458]:
features_Xtrain = df_train.columns.to_list()
features_Xtrain.remove("tip")

In [459]:
features_Xtest = df_test.columns.to_list()
features_Xtest.remove("tip")

In [460]:
X_train, y_train = df_train[features_Xtrain], df_train["tip"]

In [461]:
X_test, y_test = df_test[features_Xtest], df_test.pop("tip")

- Mittels einer 5-fachen Kreuzvaliedierung und Hyperparameteroptimierung wird die Beste Kombination zwischen Baumtiefe (max_depth) und minimaler Anzahl an benötigter Beispiele für einen Split (min_samples_split) gesucht

In [462]:
gs_crossv = GridSearchCV(
 estimator= DecisionTreeClassifier(),
 param_grid= dict(min_samples_split= [2, 10, 20, 50, 100], max_depth= [1, 3, 6], random_state= [0]),
 scoring = "accuracy",
 n_jobs= -1,
 cv= 5
).fit(X_train, y_train)

print(f"Best parameters: {gs_crossv.best_params_}, with score: {gs_crossv.best_score_}")

Best parameters: {'max_depth': 6, 'min_samples_split': 2, 'random_state': 0}, with score: 0.7824202903027968


In [463]:
dtc2 = DecisionTreeClassifier(min_samples_split= 2, max_depth= 6, random_state= 0).fit(X_train, y_train)

y_pred_train_best_dtc2 = dtc2.predict(X_train)
y_pred_test_best_dtc2 = dtc2.predict(X_test)

print_evaluation("DTC", X_train, X_test, y_train, y_test, y_pred_train_best_dtc2, y_pred_test_best_dtc2)

DTC Evaluation:
              ACC |            REC |        PRE |     rows |  columns
Train     0.78948 |        0.76055 |    0.77558 |   551807 |      101
Test      0.75044 |        0.63784 |    0.66892 |    58042 |      101



**Das Modell schneidet in der Evaluation besser ab, als das Basismodell ohne Feature Engineering in P3**

In [464]:
print(f"Feature Importance: {dict(zip(X_train.columns, dtc2.feature_importances_))}")

Feature Importance: {'order_id': 2.5400740965505783e-06, 'product_id': 0.0, 'add_to_cart_order': 2.2552036253414106e-05, 'reordered': 0.0, 'user_id': 0.0, 'order_number': 3.95697289022033e-05, 'order_dow': 0.03991272234912663, 'order_hour_of_day': 0.017466965833130243, 'aisle_id': 0.0, 'department_id': 0.0, 'user_tip_ratio': 0.942555649978491, 'county_Alameda': 0.0, 'county_Alpine': 0.0, 'county_Amador': 0.0, 'county_Calaveras': 0.0, 'county_Colusa': 0.0, 'county_Contra Costa': 0.0, 'county_Del Norte': 0.0, 'county_El Dorado': 0.0, 'county_Fresno': 0.0, 'county_Glenn': 0.0, 'county_Humboldt': 0.0, 'county_Imperial': 0.0, 'county_Inyo': 0.0, 'county_Kern': 0.0, 'county_Kings': 0.0, 'county_Lake': 0.0, 'county_Lassen': 0.0, 'county_Los Angeles': 0.0, 'county_Madera': 0.0, 'county_Marin': 0.0, 'county_Mariposa': 0.0, 'county_Mendocino': 0.0, 'county_Merced': 0.0, 'county_Modoc': 0.0, 'county_Mono': 0.0, 'county_Monterey': 0.0, 'county_Napa': 0.0, 'county_Nevada': 0.0, 'county_Orange': 0.0

In [465]:
feature_importances_p4 = pd.DataFrame.from_dict(dict(zip(X_train.columns, dtc2.feature_importances_)), orient= "index", columns= ["feature_importance"])

feature_importances_sorted = feature_importances_p4.sort_values(by= "feature_importance", ascending= True).tail(9)
df_feature_imp_p4 = feature_importances_sorted.reset_index().rename(columns= {"index": "feature"})

In [466]:
df_feature_imp_p4

,feature,feature_importance
0,county_Marin,0.000000
1,county_Madera,0.000000
2,county_Nevada,0.000000
3,order_id,0.000003
4,add_to_cart_order,0.000023
5,order_number,0.000040
6,order_hour_of_day,0.017467
7,order_dow,0.039913
8,user_tip_ratio,0.942556


In [467]:
# P4 Plot: Feature Importance

fig = go.Figure()


for feature in df_feature_imp_p4.feature.unique():

 if df_feature_imp_p4.loc[df_feature_imp_p4.feature == feature].feature_importance.values >= 0.1:
  color= px.colors.sequential.Purpor[5]

  fig.add_trace(
   go.Bar(
    x= df_feature_imp_p4.loc[df_feature_imp_p4.feature == feature].feature_importance,
    y= df_feature_imp_p4.loc[df_feature_imp_p4.feature == feature].feature,
    marker= dict(color= color),
    orientation = "h",
    text= df_feature_imp_p4.loc[df_feature_imp_p4.feature == feature].feature_importance.round(2),
    showlegend= False,
    width= 0.8,
    hovertemplate= f"<b>{feature}</b><br><br>" +
                   "feature_importance: %{x}<br>" +
                   "<extra></extra>"
    )
  )
 else:
  color= px.colors.sequential.Greys[3]

  fig.add_trace(
   go.Bar(
    x= df_feature_imp_p4.loc[df_feature_imp_p4.feature == feature].feature_importance,
    y= df_feature_imp_p4.loc[df_feature_imp_p4.feature == feature].feature,
    marker= dict(color= color),
    orientation = "h",
    text= df_feature_imp_p4.loc[df_feature_imp_p4.feature == feature].feature_importance.round(3),
    showlegend= False,
    width= 0.8,
    hovertemplate= f"<b>{feature}</b><br><br>" +
                   "feature_importance: %{x}<br>" +
                   "<extra></extra>"
    )
  )

fig.update_layout(title= dict(text= "<b>Feature Importance der Attribute</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), height= 600)
fig.update_xaxes(title_text="<b>Importance</b>", ticks= "outside")
fig.update_xaxes(showline= True)
fig.update_yaxes(title_text="<b>Feature</b>", ticks= "outside", showgrid= True, gridcolor= "grey", showline= True)
fig.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))
fig.show()

- feature_importance von user_tip_ratio mit 0.9426 am höchsten
 
- **damit ist die Tip-Historie eines Users auschschlaggebend für die Tippwahrscheinlichkeit bei seiner aktuellen Bestellung** 

## 3. Zusätzliche Fragestellungen:

#### Machine Learning Modell:

##### Preprocessing und Feature Engineering:

In [468]:
df_g = df.copy()

**Nan-values, welche in der Spalte days_since_prior_orders auftreten wenn es die erste Bestellung eines Kunden ist, für die Methode Decision Tree Classifier mit 0 auffüllen**

In [469]:
df = df_g.reset_index(drop= True)

**Sampling des Datensatzes um Rechenzeit zu sparen**

In [470]:
df_sample = df.sample(frac= 0.1)

**Erzeugen des Features user_tip_raito**

In [471]:
df_tip = df_sample.groupby(["user_id", "order_id", "order_number"], as_index= False).agg(tip = ("tip", "mean")).sort_values(by= ["user_id", "order_number"], ascending= True)
df_tip["tip_cumsum"] = df_tip.groupby("user_id", as_index= False)["tip"].cumsum()
df_tip["user_tip_ratio"] = df_tip.tip_cumsum / df_tip.order_number

df_tip_merge = df_tip[["user_id", "order_id", "order_number", "user_tip_ratio"]]
df_sample = df_sample.merge(df_tip_merge, on = ["user_id", "order_id", "order_number"], how= "right")

In [472]:
df_classifier = df_sample.copy()

**One-Hot-Encoding der Spalten County und Days_since_prior_order**

- HistogramGradientBoostingClassifier kann Nan-Values verarbeiten weshalb nur die Spalte County One-Hot Encoded wird

In [473]:
df_classifier_dtc = pd.get_dummies(df_classifier, columns= ["county", "days_since_prior_order"], dummy_na= True)

In [474]:
df_classifier_ghbc = pd.get_dummies(df_classifier, columns= ["county"])

In [475]:
X_dtc = df_classifier_dtc.drop(["product_name", "department", "aisle", "tip"], axis= 1)
y_dtc = df_classifier_dtc["tip"]

X_ghbc = df_classifier_ghbc.drop(["product_name", "department", "aisle", "tip"], axis= 1)
y_ghbc = df_classifier_dtc["tip"]

**Trainings- und Testdatensplit**

In [476]:
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_dtc, y_dtc, test_size= 0.2, random_state=0)

In [477]:
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(X_ghbc, y_ghbc, test_size= 0.2, random_state=0)

##### **HGBC**

In [478]:
gs_crss_hgbc = GridSearchCV(
 estimator= HistGradientBoostingClassifier(),
 param_grid= dict(max_iter= [100], learning_rate= [0.001, 0.01, 0.1], max_depth= [1,3,5,9], random_state= [0]),
 scoring= "accuracy",
 n_jobs= -1
).fit(X_train_h, y_train_h)

print(f"Best param: {gs_crss_hgbc.best_params_} with acc: {gs_crss_hgbc.best_score_}")

Best param: {'learning_rate': 0.1, 'max_depth': 9, 'max_iter': 100, 'random_state': 0} with acc: 0.8023554223826659


In [479]:
best_hgbc = HistGradientBoostingClassifier(learning_rate= 0.1, max_depth= 9, max_iter= 100, random_state= 0).fit(X_train_h, y_train_h)

y_best_train_pred = best_hgbc.predict(X_train_h)
y_best_test_pred = best_hgbc.predict(X_test_h)

print_evaluation("HGBC", X_train_h, X_test_h, y_train_h, y_test_h, y_best_train_pred, y_best_test_pred)

HGBC Evaluation:
              ACC |            REC |        PRE |     rows |  columns
Train     0.80463 |        0.80263 |    0.77061 |   490613 |       69
Test      0.80079 |        0.79676 |    0.76613 |   122654 |       69



##### **DTC**

In [480]:
gs_crss_dtc = GridSearchCV(
 estimator= DecisionTreeClassifier(),
 param_grid= dict(min_samples_split= [2, 10, 20, 50, 100], max_depth= [1, 3, 5, 9], random_state= [0]),
 scoring= "accuracy",
 n_jobs= -1
).fit(X_train_d, y_train_d)

print(f"Best param: {gs_crss_dtc.best_params_} with acc: {gs_crss_dtc.best_score_}")

Best param: {'max_depth': 9, 'min_samples_split': 2, 'random_state': 0} with acc: 0.7974717358352487


In [481]:
best_dtc = DecisionTreeClassifier(max_depth= 9, min_samples_split= 2, random_state= 0).fit(X_train_d, y_train_d)

y_best_train_pred = best_dtc.predict(X_train_d)
y_best_test_pred = best_dtc.predict(X_test_d)

print_evaluation("DTC", X_train_d, X_test_d, y_train_d, y_test_d, y_best_train_pred, y_best_test_pred)

DTC Evaluation:
              ACC |            REC |        PRE |     rows |  columns
Train     0.80131 |        0.79246 |    0.77041 |   490613 |      101
Test      0.79847 |        0.78768 |    0.76697 |   122654 |      101



- Das Klassifikationsmodell mit der Methodik HistogramGradientBoostingClassifier schneidet in der Evaluation besser als der DecisionTreeClassifier ab  
 
- Allgemein konnten wir mit einer Genauigkeit von 80.4% auf den Testdaten hervorsagen ob eine Bestellung ein Trinkgeld erhält oder nicht 